In [ ]:
# Cell 0: Import libraries & define model class
import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import pandas as pd
from typing import List, Dict
from sklearn.preprocessing import StandardScaler

# Define model class FIRST
class DiabetesNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Load data
df = pd.read_csv('../data/diabetes.csv')
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
test_ds = TensorDataset(X_test_t, y_test_t)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

print("Setup complete - data loaded, model defined.")

# Now pre-train global model
global_model = DiabetesNet()

criterion = nn.BCELoss()
optimizer = optim.Adam(global_model.parameters(), lr=0.001)

train_ds_global = TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                                torch.tensor(y_train, dtype=torch.float32).unsqueeze(1))
train_loader_global = DataLoader(train_ds_global, batch_size=32, shuffle=True)

epochs =12
for epoch in range(epochs):
    global_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_global:
        optimizer.zero_grad()
        outputs = global_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Pre-train Epoch {epoch+1}: Loss {running_loss / len(train_loader_global):.4f}")

# Extract initial parameters
initial_parameters = [p.detach().cpu().numpy() for p in global_model.parameters()]

print("Pre-trained global model ready. Initial parameters extracted.")
print(f"Number of parameter tensors: {len(initial_parameters)}")

Setup complete - data loaded, model defined.
Pre-train Epoch 1: Loss 0.6526
Pre-train Epoch 2: Loss 0.6416
Pre-train Epoch 3: Loss 0.6334
Pre-train Epoch 4: Loss 0.6122
Pre-train Epoch 5: Loss 0.5863
Pre-trained global model ready. Initial parameters extracted.
Number of parameter tensors: 6


In [31]:
import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import pandas as pd
from typing import List, Dict

# Reload your data and scaler if needed (copy from previous notebook)
df = pd.read_csv('../data/diabetes.csv')
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

# Assuming you already have scaler from baseline
# If not, re-fit it here:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on full X for consistency (or just train part)
X_scaled = scaler.fit_transform(X)

# Use your previous train/test split (or re-create for reproducibility)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
test_ds = TensorDataset(X_test_t, y_test_t)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

# Your model class (copy from baseline)
class DiabetesNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

print("Setup complete - data loaded, model defined.")

Setup complete - data loaded, model defined.


In [32]:
def set_parameters(net: nn.Module, parameters: List[np.ndarray]):
    """Load numpy parameters into model state_dict"""
    if not parameters:
        print("Warning: Empty parameters received - skipping load")
        return
    
    state_dict = net.state_dict()
    params_dict = zip(state_dict.keys(), parameters)
    
    for key, param in params_dict:
        state_dict[key] = torch.from_numpy(param).float()
    
    net.load_state_dict(state_dict, strict=True)  # Changed to strict=True
    print(f"[DEBUG] Parameters loaded into model")

def get_parameters(net: nn.Module) -> List[np.ndarray]:
    """Extract parameters as numpy arrays"""
    params = [val.cpu().detach().numpy() for val in net.parameters()]
    print(f"[DEBUG] Extracted {len(params)} parameter tensors")
    return params

def train_local(net: nn.Module, trainloader: DataLoader, epochs: int = 1, device: str = "cpu"):
    """Train model locally and return number of samples trained"""
    criterion = nn.BCELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    net.to(device)
    net.train()
    
    total_samples = 0
    total_loss = 0.0
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for data, target in trainloader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = net(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * target.size(0)
            total_samples += target.size(0)
        
        avg_epoch_loss = epoch_loss / len(trainloader.dataset)
        print(f"  Local training epoch loss: {avg_epoch_loss:.4f}")
    
    return total_samples

def evaluate_local(net: nn.Module, testloader: DataLoader, device: str = "cpu"):
    criterion = nn.BCELoss()
    loss = 0.0
    correct, total = 0, 0
    net.eval()
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = net(data)
            loss += criterion(output, target).item() * target.size(0)
            pred = (output > 0.5).float()
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    loss /= total
    accuracy = correct / total
    return loss, accuracy

In [ ]:
class DiabetesClient(fl.client.NumPyClient):
    def __init__(self, cid: str, net: nn.Module, trainloader: DataLoader, valloader: DataLoader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        try:
            return get_parameters(self.net)
        except Exception as e:
            print(f"Client {self.cid} failed get_parameters: {e}")
            raise

    def fit(self, parameters, config):
        print(f"\n[Client {self.cid}] fit() called with {len(parameters)} params")
        set_parameters(self.net, parameters)
        
        # Get params before training
        before_params = [p.detach().cpu().numpy().copy() for p in self.net.parameters()]
        
        num_samples = train_local(self.net, self.trainloader, epochs=12)
        
        # Get params after training
        after_params = get_parameters(self.net)
        
        # Check if params changed
        param_changed = False
        for i, (before, after) in enumerate(zip(before_params, after_params)):
            if not np.allclose(before, after):
                param_changed = True
                print(f"[Client {self.cid}] Parameter {i} changed: max diff = {np.max(np.abs(before - after)):.6f}")
        
        if not param_changed:
            print(f"[Client {self.cid}] WARNING: No parameters changed during training!")
        
        print(f"[Client {self.cid}] fit() returning {num_samples} samples trained")
        return get_parameters(self.net), num_samples, {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = evaluate_local(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

# Updated client_fn with proper context handling
from flwr.common import Context

def client_fn(context: Context) -> fl.client.Client:
    """Flower client constructor - creates a new client per partition"""
    # Extract client ID - try multiple approaches for compatibility
    try:
        cid = str(context.client_id)
    except:
        try:
            cid = str(context.node_config.get("partition-id", 0))
        except:
            cid = "0"
    
    cid_int = int(cid) if cid.isdigit() else 0
    
    # Ensure cid_int is within valid range
    if cid_int >= NUM_CLIENTS:
        cid_int = cid_int % NUM_CLIENTS
    
    print(f"Creating client {cid_int} with {len(trainloaders[cid_int].dataset)} train samples")
    
    # Create fresh model and client for this partition
    net = DiabetesNet()
    client = DiabetesClient(
        cid=str(cid_int),
        net=net,
        trainloader=trainloaders[cid_int],
        valloader=valloaders[cid_int]
    )
    return client.to_client()

In [34]:
NUM_CLIENTS = 10

# Split training data into NUM_CLIENTS parts (simulate different clinics)
X_train_splits = np.array_split(X_train, NUM_CLIENTS)
y_train_splits = np.array_split(y_train, NUM_CLIENTS)

trainloaders = []
valloaders = []

for i in range(NUM_CLIENTS):
    # Safe split without stratify
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train_splits[i], y_train_splits[i],
        test_size=0.2, 
        random_state=42 + i
    )
    
    # Optional: print to debug
    print(f"Client {i}: train samples={len(y_tr)}, val samples={len(y_val)}, "
          f"pos class in train: {np.mean(y_tr):.2%}")
    
    train_ds = TensorDataset(torch.tensor(X_tr, dtype=torch.float32), 
                             torch.tensor(y_tr, dtype=torch.float32).unsqueeze(1))
    val_ds   = TensorDataset(torch.tensor(X_val, dtype=torch.float32), 
                             torch.tensor(y_val, dtype=torch.float32).unsqueeze(1))
    
    trainloaders.append(DataLoader(train_ds, batch_size=16, shuffle=True))
    valloaders.append(DataLoader(val_ds, batch_size=16, shuffle=False))

print(f"Created {NUM_CLIENTS} clients with local train/val splits.")

Client 0: train samples=49, val samples=13, pos class in train: 38.78%
Client 1: train samples=49, val samples=13, pos class in train: 36.73%
Client 2: train samples=49, val samples=13, pos class in train: 30.61%
Client 3: train samples=49, val samples=13, pos class in train: 34.69%
Client 4: train samples=48, val samples=13, pos class in train: 35.42%
Client 5: train samples=48, val samples=13, pos class in train: 39.58%
Client 6: train samples=48, val samples=13, pos class in train: 37.50%
Client 7: train samples=48, val samples=13, pos class in train: 41.67%
Client 8: train samples=48, val samples=13, pos class in train: 37.50%
Client 9: train samples=48, val samples=13, pos class in train: 35.42%
Created 10 clients with local train/val splits.


In [35]:
import ray
ray.init(ignore_reinit_error=True, num_cpus=4)
print("Ray initialized manually.")

2026-02-07 16:51:19,658	INFO worker.py:1850 -- Calling ray.init() again after it has already been called.


Ray initialized manually.


In [36]:
import flwr as fl
print("Flower version:", fl.__version__)

Flower version: 1.26.1


In [ ]:
# Cell 5: Define Client Function & Run Simulation (FIXED)

# Strategy with explicit metric aggregation
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(initial_parameters),
    evaluate_metrics_aggregation_fn=lambda results: {
        "accuracy": np.mean([r["accuracy"] for _, r in results]) if results else 0.0
    }
)

print("Starting federated simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources={"num_cpus": 2, "num_gpus": 0.0},  # Increased CPU resources
)

print("Simulation finished.")
if hasattr(history, 'metrics_distributed'):
    print("History distributed metrics:", history.metrics_distributed)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Starting federated simulation...


2026-02-07 16:51:30,859	INFO worker.py:2012 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 628235059.0, 'node:127.0.0.1': 1.0, 'memory': 1465881805.0, 'node:__internal_head__': 1.0, 'CPU': 4.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=14816) Creating client 9 with 48 train samples
(ClientAppActor pid=14816) 
(ClientAppActor pid=14816) [Client 9] fit() called with 6 params
(ClientAppActor pid=14816) [DEBUG] Parameters loaded into model
(ClientAppActor pid=22224) 


(pid=gcs_server) [2026-02-07 16:51:56,509 E 25012 1368] (gcs_server.exe) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


...

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=14816) 
(ClientAppActor pid=22224) 


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 19.94s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.5224156111478806
INFO :      		round 2: 0.5089602440595626
INFO :      		round 3: 0.49736054241657257
INFO :      		round 4: 0.4875556528568268
INFO :      		round 5: 0.47851642668247224
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, np.float64(0.8)),
INFO :      	              (2, np.float64(0.8)),
INFO :      	              (3, np.float64(0.8076923076923078)),
INFO :      	              (4, np.float64(0.8307692307692308)),
INFO :      	              (5, np.float64(0.823076923076923))]}
INFO :      


Simulation finished.
History distributed metrics: {'accuracy': [(1, np.float64(0.8)), (2, np.float64(0.8)), (3, np.float64(0.8076923076923078)), (4, np.float64(0.8307692307692308)), (5, np.float64(0.823076923076923))]}
(ClientAppActor pid=14816) Creating client 0 with 49 train samples
(ClientAppActor pid=14816) [DEBUG] Parameters loaded into model


(ClientAppActor pid=22224) [2026-02-07 16:52:19,638 E 22224 18804] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


In [38]:
# Cell 6: Inspect History & Evaluate Global Model

print("Simulation History:")
print(history)  # Should show loss, accuracy per round if metrics collected

# If history.metrics_centralized or distributed available
if hasattr(history, 'metrics_centralized'):
    print("Centralized metrics:", history.metrics_centralized)
if hasattr(history, 'metrics_distributed'):
    print("Distributed metrics:", history.metrics_distributed)

# To evaluate the final global model properly, we need the aggregated parameters.
# For simplicity: re-create global model and assume last aggregated params (manual extract from strategy if needed).
# Quick hack: Run evaluation on test set using a model trained centrally for comparison (or note round accuracies from logs)

# From logs, look for lines like:
# evaluate_round 5 aggregated results: {'accuracy': X.XX}
# If you see them in output, note the last one (e.g., round 5 accuracy)

# Bonus: Print client accuracies from logs if visible

Simulation History:
History (loss, distributed):
	round 1: 0.5224156111478806
	round 2: 0.5089602440595626
	round 3: 0.49736054241657257
	round 4: 0.4875556528568268
	round 5: 0.47851642668247224
History (metrics, distributed, evaluate):
{'accuracy': [(1, np.float64(0.8)),
              (2, np.float64(0.8)),
              (3, np.float64(0.8076923076923078)),
              (4, np.float64(0.8307692307692308)),
              (5, np.float64(0.823076923076923))]}

Centralized metrics: {}
Distributed metrics: {'accuracy': [(1, np.float64(0.8)), (2, np.float64(0.8)), (3, np.float64(0.8076923076923078)), (4, np.float64(0.8307692307692308)), (5, np.float64(0.823076923076923))]}
